# 上傳檔案到 NodeMCU (Upload files to NodeMCU) - LORA

### 需先安裝 ampy ( Adafruit MicroPython Tool )
#### pip install adafruit-ampy
https://github.com/adafruit/ampy  

[Blog](https://wei1234c.blogspot.tw/2017/08/sx127x-lora-transceiver-driver-for.html)  

In [20]:
import os
import shutil

### 設定COM port (set current COM port), baud rate

In [93]:
baud_rate = 115200
# baud_rate = 230400
# baud_rate = 460800
# baud_rate = 921600

com_port = 'COM7'
# com_port = 'COM12'
# com_port = 'COM13'
com_port = 'COM15'
# com_port = 'COM16'
# com_port = 'COM17'
# com_port = '/dev/ttyUSB0'

### Utility functions

In [22]:
def clear_local_folder(folder):
    print('\n[Clearing folder {}]'.format(folder))
    for file in os.listdir(folder):
        os.remove(os.path.join(folder, file))
        
        
def copy_one_file_to_local_folder(folder, file, destination_folder):
    print('Copying {} to {}'.format(file, destination_folder))
    shutil.copy(os.path.join(folder, file), destination_folder)
    
    
def copy_all_files_to_local_folder(folders, destination_folder):
    print('\n[Copying all files to upload folder {}]'.format(destination_folder))
    clear_local_folder(destination_folder)    
    
    for folder in folders: 
        for file in os.listdir(folder):
            if (file.endswith('.py') or file.endswith('.mpy')) and not file.startswith('_'):
                copy_one_file_to_local_folder(folder, file, destination_folder)

In [23]:
def list_files_in_device(com_port):
    files = !ampy --port {com_port} --baud {baud_rate} ls
    return sorted(files)


def cat_file_from_device(com_port, file):
    !ampy --port {com_port} --baud {baud_rate} get {file}
    

def delete_file_in_device(com_port, file):
    print('Deleting {}'.format(file))
    !ampy --port {com_port} --baud {baud_rate} rm {file}
    

def delete_files_in_device(com_port):
    print('\n[Deleting all files in device {}]'.format(com_port))
    for file in list_files_in_device(com_port):
        delete_file_in_device(com_port, file)
        
#     try:
#         !ampy --port {com_port} --baud {baud_rate} rmdir {'/'}
#     except Exception as e:
#         print(e)        
        
        
def copy_one_file_to_device(com_port, folder, file, mpy_only = False):
    if mpy_only:
        if file.endswith('.mpy'):
            print('Copying {}'.format(file))
            !ampy --port {com_port} --baud {baud_rate} put {os.path.join(folder, file)}  
    elif file.endswith('.py'):
        print('Copying {}'.format(file))
        !ampy --port {com_port} --baud {baud_rate} put {os.path.join(folder, file)}   
        

def delet_main_in_device(com_port, main_file_names):
    print('Deleting {}'.format(main_file_names))
    files = list_files_in_device(com_port)

    for file in main_file_names:
        if file in files:
            delete_file_in_device(com_port, file)
            
            
def delete_main_and_files_in_device(com_port, main_file_names = ['main.py', 'main.mpy'], delete_first = True, mpy_only = False):
    print('\n[Copying all files to device {}]'.format(com_port))
    delet_main_in_device(com_port, main_file_names)
    if mpy_only: delete_first = True
    if delete_first: delete_files_in_device(com_port)
        

def copy_all_files_to_device(com_port, folder, main_file_names = ['main.py', 'main.mpy'], delete_first = True, mpy_only = False):    
    for file in os.listdir(folder):
        if (file.endswith('.py') or file.endswith('.mpy')) and not file.startswith('_') and not file in main_file_names:
            copy_one_file_to_device(com_port, folder, file, mpy_only)                    
    
    for file in main_file_names:
        if os.path.isfile(os.path.join(folder, file)):
            copy_one_file_to_device(com_port, folder, file, mpy_only)  
            
            
def do_all_to_device(com_port, folder, main_file_names = ['main.py', 'main.mpy'], delete_first = True, mpy_only = False):
    delete_main_and_files_in_device(com_port, main_file_names, delete_first, mpy_only)
    copy_all_files_to_device(com_port, folder, main_file_names, delete_first, mpy_only)

### Copy *.py to device (all needed fils will be put in the same folder)

In [91]:
folders = [os.path.sep.join(['..', '..', 'codes', 'micropython']),
           os.path.sep.join(['..', '..', 'codes', 'demo']),
           os.path.sep.join(['..', '..', 'codes', 'sx127x']),]

upload_folder = os.path.sep.join(['upload', 'py'])

copy_all_files_to_local_folder(folders, upload_folder)


[Copying all files to upload folder upload\py]

[Clearing folder upload\py]
Copying boot.py to upload\py
Copying config.py to upload\py
Copying controller_esp.py to upload\py
Copying controller_rpi.py to upload\py
Copying led.py to upload\py
Copying main.py to upload\py
Copying LoRaDumpRegisters.py to upload\py
Copying LoRaDuplex.py to upload\py
Copying LoRaDuplexCallback.py to upload\py
Copying LoRaReceiver.py to upload\py
Copying LoRaReceiverCallback.py to upload\py
Copying LoRaSender.py to upload\py
Copying LoRaSetSpread.py to upload\py
Copying LoRaSetSyncWord.py to upload\py
Copying test.py to upload\py
Copying sx127x.py to upload\py


In [90]:
upload_folder_py = os.path.sep.join(['upload', 'py'])

do_all_to_device(com_port, upload_folder, delete_first = True)

print('\n[All done!]')


[Copying all files to device COM17]
Deleting ['main.py', 'main.mpy']
Deleting main.py

[Deleting all files in device COM17]
Deleting LoRaDumpRegisters.py
Deleting LoRaDuplex.py
Deleting LoRaDuplexCallback.py
Deleting LoRaReceiver.py
Deleting LoRaReceiverCallback.py
Deleting LoRaSender.py
Deleting LoRaSetSpread.py
Deleting LoRaSetSyncWord.py
Deleting boot.py
Deleting config.py
Deleting controller_esp.py
Deleting controller_rpi.py
Deleting led.py
Deleting sx127x.py
Deleting test.py
Copying boot.py
Copying config.py
Copying controller_esp.py
Copying controller_rpi.py
Copying led.py
Copying LoRaDumpRegisters.py
Copying LoRaDuplex.py
Copying LoRaDuplexCallback.py
Copying LoRaReceiver.py
Copying LoRaReceiverCallback.py
Copying LoRaSender.py
Copying LoRaSetSpread.py
Copying LoRaSetSyncWord.py
Copying sx127x.py
Copying test.py
Copying main.py

[All done!]


### Copy *.mpy to device

In [ ]:
upload_folder_py = os.path.sep.join(['upload', 'py'])
upload_folder_mpy = os.path.sep.join(['upload', 'mpy'])

do_all_to_device(com_port, upload_folder_mpy, mpy_only = True)

copy_one_file_to_device(com_port, upload_folder_py, 'boot.py')
copy_one_file_to_device(com_port, upload_folder_py, 'main.py')

print('\n[All done!]')

### 單一檔案上傳 (single file upload, in case needed)

In [81]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'micropython']), 'controller_esp.py') 

Copying controller_esp.py


In [66]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'micropython']), 'config.py') 

Copying config.py


In [94]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'sx127x']), 'sx127x.py') 

Copying sx127x.py


In [39]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'demo']), 'test.py') 

Copying test.py


In [50]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'demo']), 'LoRaSender.py') 

Copying LoRaSender.py


In [165]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'demo']), 'LoRaReceiver.py') 

Copying LoRaReceiver.py


In [187]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'demo']), 'LoRaSetSpread.py') 

Copying LoRaSetSpread.py


In [44]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'demo']), 'LoRaSetSyncWord.py') 

Copying LoRaSetSyncWord.py


In [94]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'demo']), 'LoRaReceiverCallback.py') 

Copying LoRaReceiverCallback.py


In [109]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'demo']), 'LoRaDuplex.py') 

Copying LoRaDuplex.py


In [14]:
copy_one_file_to_device(com_port, os.path.sep.join(['..', '..', 'codes', 'demo']), 'LoRaDuplexCallback.py') 

Copying LoRaDuplexCallback.py


### 列出檔案 (list files)

In [ ]:
# list_files_in_device(com_port)

### 檢查檔案內容 (check file content)

In [29]:
# cat_file_from_device(com_port, 'sx127x.py')

### 連網測試 (network config and test)

In [ ]:
# 連上網路
# import network; nic=network.WLAN(network.STA_IF); nic.active(False);  # disable network
# import network; nic=network.WLAN(network.STA_IF); nic.active(True); nic.connect('SSID','password');nic.ifconfig()
# import network; nic=network.WLAN(network.STA_IF); nic.active(True); nic.connect('Kingnet-70M-$370', '');nic.ifconfig()
# import network; nic=network.WLAN(network.STA_IF); nic.ifconfig()
# import network; nic=network.WLAN(network.STA_IF);nic.ifconfig();nic.config('mac');nic.ifconfig((['mac',])

In [ ]:
# 發出 http request
# import socket;addr=socket.getaddrinfo('micropython.org',80)[0][-1]
# s = socket.socket();s.connect(addr);s.send(b'GET / HTTP/1.1\r\nHost: micropython.org\r\n\r\n');data = s.recv(1000);s.close()

In [ ]:
# Delete all files
# import u_python;u_python.del_all_files();import os;os.listdir()


### Run Own Broker container on Raspberry Pi
copy folder 'codes' to Raspberry Pi under folder '/data/elastic_network_of_things_with_micropython',  
so Raspberry Pi has folder '/data/elastic_network_of_things_with_micropython/codes'  
then run the command below on Raspberry Pi.

```
docker run -it -p 9662:9662 --name=Broker --hostname=Broker --volume=/data/elastic_network_of_things_with_micropython:/project wei1234c/python_armv7 /bin/sh -c "cd /project/codes/broker && python3 broker.py" 
```

### Run MQTT Broker container on Raspberry Pi  

https://store.docker.com/community/images/pascaldevink/rpi-mosquitto

place your mosquitto.conf in /srv/mqtt/config/  
NOTE: You have to change the permissions of the directories  
to allow the user to read/write to data and log and read from  

config directory
For TESTING purposes you can use chmod -R 777 /srv/mqtt/*  
Better use "-u" with a valid user id on your docker host  

on Raspberry Pi  
copy mqtt/config mqtt/log mqtt/data on to RPi under /data/elastic_network_of_things_with_mqtt_and_micropython/mqtt  

grand permission for user  
sudo chmod -R 777 /data/elastic_network_of_things_with_mqtt_and_micropython/mqtt/*  
```
docker run -it --rm -p 1883:1883 -p 9001:9001 --name=Mosquitto --hostname=Mosquitto \  
--volume=/data/elastic_network_of_things_with_mqtt_and_micropython/mqtt/config:/mqtt/config:ro \
--volume=/data/elastic_network_of_things_with_mqtt_and_micropython/mqtt/data:/mqtt/data \
--volume=/data/elastic_network_of_things_with_mqtt_and_micropython/mqtt/log:/mqtt/log \
pascaldevink/rpi-mosquitto /usr/sbin/mosquitto -v -c /mqtt/config/mosquitto.conf
```